In [138]:
import psycopg2
import pandas as pd
import psycopg2
from IPython.display import display


host = "host"
port = "5432"
database = "db"
user = "username"
password = "passowrd"

try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar: {e}")


Erro ao conectar: 


In [137]:
# Crie um cursor para executar comandos SQL no banco de dados
cursor = conn.cursor()

 # Crie um cursor para executar comandos SQL no banco de dados
# Execute a consulta SQL para juntar as tabelas e obter os dados desejados
cursor.execute("""
    SELECT 
        candidates.cnd_id, 
        candidates.cnd_age, 
        candidates.cnd_name, 
        candidates.cnd_current_profession, 
        candidates.cnd_experiences, 
        skl_skill.skl_name as skill_name, 
        skl_skill.skl_description as skill_description, 
        knw_knowledge.knw_name as knowledge_name, 
        knw_knowledge.knw_description as knowledge_description, 
        att_attitude.att_name as attitude_name, 
        att_attitude.att_description as attitude_description
    FROM candidates
    INNER JOIN candidate_skill ON candidates.cnd_id = candidate_skill.cnd_id
    INNER JOIN skl_skill ON candidate_skill.knw_id = skl_skill.skl_id
    INNER JOIN knw_knowledge ON skl_skill.skl_id = knw_knowledge.knw_id
    INNER JOIN candidate_attitude ON candidates.cnd_id = candidate_attitude.cnd_id
    INNER JOIN att_attitude ON candidate_attitude.knw_id = att_attitude.att_id
""")


# Recupereatodos os registros armazena em um DataFrame
df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
display(df)

cursor.close()




,cnd_id,cnd_age,cnd_name,cnd_current_profession,cnd_experiences,skill_name,skill_description,knowledge_name,knowledge_description,attitude_name,attitude_description
0,2,20,José,dev jr,empresa xpto 3 anos,skill,dkill descr,123123,123,123,strasdxasding
